> **Apenas configurações iniciais para configuirar o ambiente**

In [5]:
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

import findspark
findspark.init()


Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [4,436 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,211 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packa

In [6]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Colab com PySpark") \
    .getOrCreate()


In [141]:
# Criando pasta para os arquivos
!mkdir -p /ifood-case/data/raw/

!mkdir -p /ifood-case/data/processed/

# Análise Exploratória (EDA)

# 1. Leitura e inspeção inicial das bases

Datasets carregados:
- offers.json → informações das ofertas (tipo, duração, canais, desconto)
- customers.json → dados dos clientes (idade, gênero, limite, data de cadastro)
- transactions.json → eventos de interação cliente/oferta

**Objetivo:**
- Entender a estrutura geral, os tipos de dados e o volume de registros iniciais.

**Conclusão:**
- As tabelas foram carregadas corretamente e os schemas foram verificados. Todas têm colunas estruturadas como esperado.

In [8]:
# Leitura dos datasets
df_offers = spark.read.json("/ifood-case/data/raw/offers.json")
df_customers = spark.read.json("/ifood-case/data/raw/profile.json")
df_transactions = spark.read.json("/ifood-case/data/raw/transactions.json")

# Primeira visualização da estrutura
print("📦 Offers:")
df_offers.show(5, truncate=False)

print("📦 Customers:")
df_customers.show(5, truncate=False)

print("📦 Transactions:")
df_transactions.show(5, truncate=False)


📦 Offers:
+----------------------------+--------------+--------+--------------------------------+---------+-------------+
|channels                    |discount_value|duration|id                              |min_value|offer_type   |
+----------------------------+--------------+--------+--------------------------------+---------+-------------+
|[email, mobile, social]     |10            |7.0     |ae264e3637204a6fb9bb56bc8210ddfd|10       |bogo         |
|[web, email, mobile, social]|10            |5.0     |4d5c57ea9a6940dd891ad53e9dbe8da0|10       |bogo         |
|[web, email, mobile]        |0             |4.0     |3f207df678b143eea3cee63160fa8bed|0        |informational|
|[web, email, mobile]        |5             |7.0     |9b98b8c7a33c4b65b9aebfe6a799e6d9|5        |bogo         |
|[web, email]                |5             |10.0    |0b1e1539f2cc45b7b9fa7c272da2e1d7|20       |discount     |
+----------------------------+--------------+--------+--------------------------------+-------

## 1.1. Verificação de valores nulos

**Objetivo:**
- Entender os tipos de dados das colunas (schema) e identificar possíveis valores nulos ou vazios.

**Conslusão:**
- credit_card_limit e gender estão com 2.175 valores nulos. Exatamente a mesma quantidade de registros. Vamos analisar mais profundamente para entender se podemos descartar esses registros ou se deveriamos demandar esforço para trata-lo.

In [10]:
# Verificando os schemas
print("📋 Schema - Offers:")
df_offers.printSchema()

print("📋 Schema - Customers:")
df_customers.printSchema()

print("📋 Schema - Transactions:")
df_transactions.printSchema()

# obs:
  # registered_on está como string (ponto de atenção)
  # time_since_test_start está como double? vamos usá-lo como proxy de tempo cronológico.
  # dois ID's com quase o mesmo nome? (offer id/offer_id)

📋 Schema - Offers:
root
 |-- channels: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- discount_value: long (nullable = true)
 |-- duration: double (nullable = true)
 |-- id: string (nullable = true)
 |-- min_value: long (nullable = true)
 |-- offer_type: string (nullable = true)

📋 Schema - Customers:
root
 |-- age: long (nullable = true)
 |-- credit_card_limit: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- id: string (nullable = true)
 |-- registered_on: string (nullable = true)

📋 Schema - Transactions:
root
 |-- account_id: string (nullable = true)
 |-- event: string (nullable = true)
 |-- time_since_test_start: double (nullable = true)
 |-- value: struct (nullable = true)
 |    |-- amount: double (nullable = true)
 |    |-- offer id: string (nullable = true)
 |    |-- offer_id: string (nullable = true)
 |    |-- reward: double (nullable = true)



In [99]:
from pyspark.sql.functions import col, sum as spark_sum
from pyspark.sql.functions import count, when,avg, expr

def check_nulls(df, name):
    print(f"🔍 Nulos na tabela {name}")
    df.select([
        spark_sum(col(c).isNull().cast("int")).alias(c)
        for c in df.columns
    ]).show()

check_nulls(df_offers, "Offers")
check_nulls(df_customers, "Customers")
check_nulls(df_transactions, "Transactions")


🔍 Nulos na tabela Offers
+--------+--------------+--------+---+---------+----------+
|channels|discount_value|duration| id|min_value|offer_type|
+--------+--------------+--------+---+---------+----------+
|       0|             0|       0|  0|        0|         0|
+--------+--------------+--------+---+---------+----------+

🔍 Nulos na tabela Customers
+---+-----------------+------+---+-------------+
|age|credit_card_limit|gender| id|registered_on|
+---+-----------------+------+---+-------------+
|  0|             2175|  2175|  0|            0|
+---+-----------------+------+---+-------------+

🔍 Nulos na tabela Transactions
+----------+-----+---------------------+-----+
|account_id|event|time_since_test_start|value|
+----------+-----+---------------------+-----+
|         0|    0|                    0|    0|
+----------+-----+---------------------+-----+



## 1.2. Verificação e tratamento dos valores da `customer`

### 1.2.1. Verificação dos valores nulos

**Objetivo:**
- Ver se os nulos em gender e credit_card_limit estão nos mesmos registros.
- Avaliar a proporção desses nulos em relação ao total.
- Entender se há alguma lógica de negócio por trás da ausência desses dados (ex: contas não finalizadas, usuários novos).

**Conslusão:**
- Esses 2.175 clientes têm os dois campos nulos simultaneamente, o que indica um mesmo tipo de perfil. Alem disso podemos ver que os registros também tem a idade discrepante;

- Eles representam apenas 12,8% da base;

- Estão incompletos para modelagem: sem gênero, sem limite, e com idade inválida;

- **A decisão foi de excluir** agora evita imputações artificiais em massa que podem distorcer análises. Além de considerar que eu nao conseguiria inferir o perfil desse tipo de cliente;


**Hipótese**:
esses clientes são cadastros incompletos, testes, ou placeholders do sistema.

In [14]:
# Total de registros
total_customers = df_customers.count()

# Verificando nulos combinados
df_customers.select(
    count("*").alias("total"),
    count(when(col("age")==118, 1)).alias("age_118"),
    count(when(col("gender").isNull(), 1)).alias("nulos_gender"),
    count(when(col("credit_card_limit").isNull(), 1)).alias("nulos_credit_limit"),
    count(when((col("gender").isNull()) & (col("credit_card_limit").isNull()), 1)).alias("nulos_ambos")
).show()




+-----+-------+------------+------------------+-----------+
|total|age_118|nulos_gender|nulos_credit_limit|nulos_ambos|
+-----+-------+------------+------------------+-----------+
|17000|   2175|        2175|              2175|       2175|
+-----+-------+------------+------------------+-----------+



In [15]:
# Amostra de clientes com dados faltando
df_customers.filter(col("gender").isNull() | col("credit_card_limit").isNull()).show(10, truncate=False)

+---+-----------------+------+--------------------------------+-------------+
|age|credit_card_limit|gender|id                              |registered_on|
+---+-----------------+------+--------------------------------+-------------+
|118|null             |null  |68be06ca386d4c31939f3a4f0e3dd783|20170212     |
|118|null             |null  |38fe809add3b4fcf9315a9694bb96ff5|20180712     |
|118|null             |null  |a03223e636434f42ac4c3df47e8bac43|20170804     |
|118|null             |null  |8ec6ce2a7e7949b1bf142def7d0e0586|20170925     |
|118|null             |null  |68617ca6246f4fbc85e91a2a49552598|20171002     |
|118|null             |null  |8974fc5686fe429db53ddde067b88302|20161122     |
|118|null             |null  |c4863c7985cf408faee930f111475da3|20170824     |
|118|null             |null  |148adfcaa27d485b82f323aaaad036bd|20150919     |
|118|null             |null  |744d603ef08c4f33af5a61c8c7628d1c|20170801     |
|118|null             |null  |2b826eba31074a059d63b0ae8f50b7d5|2

In [16]:
df_customers.filter(
    (col("age") == 118) &
    (col("credit_card_limit").isNotNull()) &
    (col("gender").isNotNull())
).count()


0

In [17]:
df_customers.groupBy("age").count().orderBy("age").show(70)


+---+-----+
|age|count|
+---+-----+
| 18|   70|
| 19|  135|
| 20|  135|
| 21|  140|
| 22|  131|
| 23|  126|
| 24|  139|
| 25|  136|
| 26|  157|
| 27|  139|
| 28|  137|
| 29|  129|
| 30|  131|
| 31|  113|
| 32|  139|
| 33|  152|
| 34|  147|
| 35|  118|
| 36|  175|
| 37|  197|
| 38|  172|
| 39|  182|
| 40|  202|
| 41|  189|
| 42|  227|
| 43|  215|
| 44|  192|
| 45|  231|
| 46|  212|
| 47|  232|
| 48|  288|
| 49|  321|
| 50|  284|
| 51|  363|
| 52|  351|
| 53|  372|
| 54|  359|
| 55|  350|
| 56|  342|
| 57|  353|
| 58|  408|
| 59|  359|
| 60|  333|
| 61|  309|
| 62|  318|
| 63|  338|
| 64|  311|
| 65|  280|
| 66|  284|
| 67|  317|
| 68|  259|
| 69|  242|
| 70|  252|
| 71|  230|
| 72|  191|
| 73|  209|
| 74|  193|
| 75|  159|
| 76|  155|
| 77|  141|
| 78|  135|
| 79|  117|
| 80|  116|
| 81|  118|
| 82|  105|
| 83|  102|
| 84|   83|
| 85|   70|
| 86|   54|
| 87|   66|
+---+-----+
only showing top 70 rows



- **O foco do projeto é distribuição de cupons baseada em perfis e comportamento**, e essas linhas não têm dados para formar perfil algum.

- Não possuem gênero, crédito, nem idade válida, ou seja, não ajudam a inferir preferências nem comportamento de consumo.

- Até o momento, não há nenhuma métr ou lógica no case que dependa de contas inativas ou incompletas.

In [70]:
df_customers_clean = df_customers.filter(
    ~((col("credit_card_limit").isNull()) &
      (col("gender").isNull()) &
      (col("age") == 118))
)

# Separa para possível uso futuro, sem levar pra análise principal
df_customers_nulls = df_customers.filter(
    (col("credit_card_limit").isNull()) |
    (col("gender").isNull()) |
    (col("age") == 118)
)

### 1.2.2. Análise estatística de colunas numéricas restantes

**Objetivo:**
- detectar valores suspeitos depois da tratativa dos dados anteriores

**Conslusão: (`AGE`)**
- Representam um grupo pequeno, mas podem ser clientes reais e que tem engajamento.

- Não há evidência clara de que são erros como os 118.

- Modelos como árvores e gradient boosting lidam bem com outliers, principalmente se a variável for contínua.

- Criar faixas de idade (age_group) para modelos de classificação (pode virar variável categórica) quando chegar a parte de engenharia de features.. (# cuidado com Multicolinearidade)

- Ou capar valores em um máximo definido (winsorization)

**Conclusão: (`card limit`)**
- Faixas balanceadas com base nos quartis, evitando viés de predominância quando chegar a parte de engenharia de features.

**Hipótese:**
- Usar regularização (se for regressão) ou árvores que dividem com base em ganho de informação.



In [71]:
# Função para calcular limites e detectar outliers em uma coluna
def calcular_outliers(df, coluna):
    q1, q3 = df.approxQuantile(coluna, [0.25, 0.75], 0.01)
    iqr = q3 - q1
    limite_inferior = q1 - 1.5 * iqr
    limite_superior = q3 + 1.5 * iqr

    print(f"📌 {coluna.upper()}")
    print(f"Q1: {q1}, Q3: {q3}, IQR: {iqr}")
    print(f"Limite inferior: {limite_inferior}, Limite superior: {limite_superior}")

    return (limite_inferior, limite_superior)


> **Ver estatísticas de `age`**

In [73]:
df_customers_clean.select("age").summary("count", "min", "max", "mean", "stddev").show()


+-------+------------------+
|summary|               age|
+-------+------------------+
|  count|             14825|
|    min|                18|
|    max|               101|
|   mean| 54.39352445193929|
| stddev|17.383705365855015|
+-------+------------------+



In [75]:
from pyspark.sql.functions import col, round

df_valores_agrupados_age = df_customers_clean \
    .withColumn("age", col("age")) \
    .groupBy("age") \
    .count() \
    .orderBy("age")

df_valores_agrupados_age.show(100)



+---+-----+
|age|count|
+---+-----+
| 18|   70|
| 19|  135|
| 20|  135|
| 21|  140|
| 22|  131|
| 23|  126|
| 24|  139|
| 25|  136|
| 26|  157|
| 27|  139|
| 28|  137|
| 29|  129|
| 30|  131|
| 31|  113|
| 32|  139|
| 33|  152|
| 34|  147|
| 35|  118|
| 36|  175|
| 37|  197|
| 38|  172|
| 39|  182|
| 40|  202|
| 41|  189|
| 42|  227|
| 43|  215|
| 44|  192|
| 45|  231|
| 46|  212|
| 47|  232|
| 48|  288|
| 49|  321|
| 50|  284|
| 51|  363|
| 52|  351|
| 53|  372|
| 54|  359|
| 55|  350|
| 56|  342|
| 57|  353|
| 58|  408|
| 59|  359|
| 60|  333|
| 61|  309|
| 62|  318|
| 63|  338|
| 64|  311|
| 65|  280|
| 66|  284|
| 67|  317|
| 68|  259|
| 69|  242|
| 70|  252|
| 71|  230|
| 72|  191|
| 73|  209|
| 74|  193|
| 75|  159|
| 76|  155|
| 77|  141|
| 78|  135|
| 79|  117|
| 80|  116|
| 81|  118|
| 82|  105|
| 83|  102|
| 84|   83|
| 85|   70|
| 86|   54|
| 87|   66|
| 88|   64|
| 89|   53|
| 90|   52|
| 91|   47|
| 92|   40|
| 93|   29|
| 94|   33|
| 95|   22|
| 96|    8|
| 97|   13|
| 98

In [76]:
# Calcular Q1, Q3, IQR e limites
lim_age = calcular_outliers(df_customers_clean, "age")

📌 AGE
Q1: 43.0, Q3: 66.0, IQR: 23.0
Limite inferior: 8.5, Limite superior: 100.5


> **Ver estatísticas de `credit_card_limit`**

In [77]:
df_customers_clean.select("credit_card_limit").summary("count", "min", "max", "mean", "stddev").show()


+-------+------------------+
|summary| credit_card_limit|
+-------+------------------+
|  count|             14825|
|    min|           30000.0|
|    max|          120000.0|
|   mean|  65404.9915682968|
| stddev|21598.299410229436|
+-------+------------------+



In [78]:
lim_credit = calcular_outliers(df_customers_clean, "credit_card_limit")


📌 CREDIT_CARD_LIMIT
Q1: 49000.0, Q3: 79000.0, IQR: 30000.0
Limite inferior: 4000.0, Limite superior: 124000.0


> **Converter registered_on para date**


In [79]:
from pyspark.sql.functions import to_date

df_customers_clean = df_customers_clean.withColumn(
    "registered_on_date",
    to_date(col("registered_on").cast("string"), "yyyyMMdd")
)


In [80]:
from pyspark.sql.functions import datediff, current_date

df_customers_clean = df_customers_clean.withColumn(
    "days_since_registration",
    datediff(current_date(), col("registered_on_date"))
)


> ***Tabela Final***

In [81]:
df_customers_clean.show()

+---+-----------------+------+--------------------+-------------+------------------+-----------------------+
|age|credit_card_limit|gender|                  id|registered_on|registered_on_date|days_since_registration|
+---+-----------------+------+--------------------+-------------+------------------+-----------------------+
| 55|         112000.0|     F|0610b486422d4921a...|     20170715|        2017-07-15|                   2853|
| 75|         100000.0|     F|78afa995795e4d85b...|     20170509|        2017-05-09|                   2920|
| 68|          70000.0|     M|e2127556f4f64592b...|     20180426|        2018-04-26|                   2568|
| 65|          53000.0|     M|389bc3fa690240e79...|     20180209|        2018-02-09|                   2644|
| 58|          51000.0|     M|2eeac8d8feae4a8ca...|     20171111|        2017-11-11|                   2734|
| 61|          57000.0|     F|aa4862eba776480b8...|     20170911|        2017-09-11|                   2795|
| 26|          4600

## 1.3. Verificação e tratamento dos valores da `offers`

**Objetivo**
- Avaliar como as colunas estão se comportando.
- Se existe algum outlier ou valor inconsistente de acordo com a regra de negocio.

**Conclusão:**
- Nenhuma coluna tem valores nulos (confirmado no início).
- **A tabela esta consistente:**
  - informational não tem desconto;
  - sem valores negativos ou suspeitos;

**Hipótese**
- Como o tipo informational não concede desconto, qualquer conversão após recebê-la provavelmente representa interesse do usuário pela marca, não por incentivo financeiro. Ou usuário que tem compras recorrentes.

- Ofertas com min_value = 0 e discount_value = 0 são iscas de engajamento e podem servir como baseline de performance em análises comparativas.

In [ ]:
df_offers.groupBy("offer_type").count().show()


+-------------+-----+
|   offer_type|count|
+-------------+-----+
|     discount|    4|
|informational|    2|
|         bogo|    4|
+-------------+-----+



> *não deveríamos ver valores em discount_value ou min_value para informational, senão há inconsistência.*

In [ ]:
df_offers.select("min_value", "duration", "discount_value").summary("count", "min", "max", "mean", "stddev").show()


+-------+-----------------+------------------+-----------------+
|summary|        min_value|          duration|   discount_value|
+-------+-----------------+------------------+-----------------+
|  count|               10|                10|               10|
|    min|                0|               3.0|                0|
|    max|               20|              10.0|               10|
|   mean|              7.7|               6.5|              4.2|
| stddev|5.831904586934796|2.3213980461973533|3.583914681524163|
+-------+-----------------+------------------+-----------------+



> *todas as ofertas usam email, e a maioria é multi-canal.*




In [ ]:
from pyspark.sql.functions import explode

df_offers.select(explode("channels").alias("channel")).groupBy("channel").count().show()


+-------+-----+
|channel|count|
+-------+-----+
| mobile|    9|
|  email|   10|
| social|    6|
|    web|    8|
+-------+-----+



In [ ]:
df_offers.filter(
    (col("offer_type") == "informational") &
    ((col("discount_value") > 0) | (col("min_value") > 0))
).show()


+--------+--------------+--------+---+---------+----------+
|channels|discount_value|duration| id|min_value|offer_type|
+--------+--------------+--------+---+---------+----------+
+--------+--------------+--------+---+---------+----------+



## 1.4. Verificação e tratamento dos valores da `transactions`

**Objetivo**
- Avaliar como os dados estão se comportando dentro da struc value.
- Verificar como cada campo aparece em diferentes tipos de evento.
- Se há inconsistências nos IDs (offer_id / _offer_id)
- Se existem valores nulos inesperados.

**Conclusão**
- offer_id é usado nos eventos:
  - offer received
  - offer viewed
- _offer_id é usado exclusivamente no evento offer completed
- Aparentemente referem-se à mesma informação (ID da oferta), mas vêm com nomes diferentes dependendo do tipo de evento.
- Agora temos uma coluna única para identificar as ofertas em todas as etapas do funil de marketing.

### 1.4.1. Extração da coluna value para análise

In [40]:
from pyspark.sql.functions import col

df_transactions_flat = df_transactions.withColumn("amount", col("value.amount")) \
    .withColumn("reward", col("value.reward")) \
    .withColumn("offer_id", col("value.offer id")) \
    .withColumn("_offer_id", col("value.offer_id")) \
    .drop("value")

df_transactions_flat.groupBy("event").count().show()

df_transactions_flat.select("event", "account_id", "offer_id", "_offer_id", "amount", "reward").show(10, truncate=False)


+---------------+------+
|          event| count|
+---------------+------+
|    transaction|138953|
| offer received| 76277|
|offer completed| 33579|
|   offer viewed| 57725|
+---------------+------+

+--------------+--------------------------------+--------------------------------+---------+------+------+
|event         |account_id                      |offer_id                        |_offer_id|amount|reward|
+--------------+--------------------------------+--------------------------------+---------+------+------+
|offer received|78afa995795e4d85b5d9ceeca43f5fef|9b98b8c7a33c4b65b9aebfe6a799e6d9|null     |null  |null  |
|offer received|a03223e636434f42ac4c3df47e8bac43|0b1e1539f2cc45b7b9fa7c272da2e1d7|null     |null  |null  |
|offer received|e2127556f4f64592b11af22de27a7932|2906b810c7d4411798c6938adc9daaa5|null     |null  |null  |
|offer received|8ec6ce2a7e7949b1bf142def7d0e0586|fafdcd668e3743c1bb461111dcafc2a4|null     |null  |null  |
|offer received|68617ca6246f4fbc85e91a2a49552598|4

> *A inteção era verificar como esta a distribuição dos ID para cada tipo de evento para considerar um possivel merge entre as colunas ou então outra abordagem*

In [41]:
# Verificar nulos por evento
df_transactions_flat.groupBy("event").agg(
    count(when(col("offer_id").isNotNull(), 1)).alias("offer_id_notnull"),
    count(when(col("_offer_id").isNotNull(), 1)).alias("_offer_id_notnull"),
    count(when(col("amount").isNotNull(), 1)).alias("amount_notnull"),
    count(when(col("reward").isNotNull(), 1)).alias("reward_notnull")
).show()

# transaction: Transação realizada pelo cliente (compra comum), realmente nao deveria ter id associado

+---------------+----------------+-----------------+--------------+--------------+
|          event|offer_id_notnull|_offer_id_notnull|amount_notnull|reward_notnull|
+---------------+----------------+-----------------+--------------+--------------+
|    transaction|               0|                0|        138953|             0|
| offer received|           76277|                0|             0|             0|
|offer completed|               0|            33579|             0|         33579|
|   offer viewed|           57725|                0|             0|             0|
+---------------+----------------+-----------------+--------------+--------------+



In [42]:
from pyspark.sql.functions import coalesce

df_transactions_flat = df_transactions_flat.withColumn(
    "final_offer_id",
    coalesce(col("offer_id"), col("_offer_id"))
)


> *Verificando se o merge funcionou e não gerou inconsistencias nos numeros*

In [43]:
from pyspark.sql.functions import count, when

df_transactions_flat.groupBy("event").agg(
    count(when(col("final_offer_id").isNotNull(), 1)).alias("final_offer_id_notnull"),
    count("*").alias("total")
).show()


# Legal parece promissor considerando que a 10 minutos atras eu estava achando que nao ia ter a menor possibilidade de replicar o filtro ✨


+---------------+----------------------+------+
|          event|final_offer_id_notnull| total|
+---------------+----------------------+------+
|    transaction|                     0|138953|
| offer received|                 76277| 76277|
|offer completed|                 33579| 33579|
|   offer viewed|                 57725| 57725|
+---------------+----------------------+------+



### 1.4.1. Analise de outliers

**Objetivo**
- Verificar se tem compra fora da realidade ou recompensa "generosa demais".


**Conclusão**
- Existem valores bem baixos com relação ao amount (va­lor da tran­sa­ção)
  - Porem aparente ser uma distribuição contínua e bem povoada entre 0.05 e 0.99
  - Frequência consistente (maioria entre 100–160 ocorrências por valor)
- Se fosse ruído, esperaria algo como:
  - Frequência muito baixa (ex: 1 ou 2 ocorrências)
  - Valores aleatórios não consecutivos

- Em contrapartida, parece ter uma distribuição assimétrica e enviesada para a esquerda com altos valores de transações
  - Eles não são necessariamente outliers errados, podendo ser casos de exceção real
  - Até 20, temos milhares de registros por valor.
  - De 21 a 40, o volume vai diminuindo de forma contínua.
  - A partir de 41, temos menos de 150 registros por valor.
  - Depois de 50, os valores ocorrem com frequência extremamente baixa (em geral, menos de 10 registros por valor).

- `reward` esta bem distribuido.



In [47]:
df_transactions_flat.select("amount").summary("count", "min", "max", "mean", "stddev").show()
df_transactions_flat.select("reward").summary("count", "min", "max", "mean", "stddev").show()

+-------+------------------+
|summary|            amount|
+-------+------------------+
|  count|            138953|
|    min|              0.05|
|    max|           1062.28|
|   mean| 12.77735615639814|
| stddev|30.250528632017126|
+-------+------------------+

+-------+------------------+
|summary|            reward|
+-------+------------------+
|  count|             33579|
|    min|               2.0|
|    max|              10.0|
|   mean| 4.904136513892611|
| stddev|2.8866468823372804|
+-------+------------------+



> *A mediana esta sendo puxada muito pra baixo. E temos um valor maximo muito alto*

In [63]:
from pyspark.sql.functions import col, round

# Arredonda os valores de amount pra facilitar análise
df_valores_agrupados = df_transactions_flat \
    .withColumn("amount_rounded", round(col("amount"), 0)) \
    .groupBy("amount_rounded") \
    .count() \
    .orderBy("amount_rounded")

df_valores_agrupados.show(100)


+--------------+------+
|amount_rounded| count|
+--------------+------+
|          null|167581|
|           0.0|  5605|
|           1.0| 14086|
|           2.0| 12146|
|           3.0|  9606|
|           4.0|  7459|
|           5.0|  5928|
|           6.0|  4826|
|           7.0|  4344|
|           8.0|  3953|
|           9.0|  3822|
|          10.0|  3832|
|          11.0|  3810|
|          12.0|  3877|
|          13.0|  3845|
|          14.0|  3880|
|          15.0|  3801|
|          16.0|  3794|
|          17.0|  3625|
|          18.0|  3492|
|          19.0|  3401|
|          20.0|  3085|
|          21.0|  3024|
|          22.0|  2795|
|          23.0|  2548|
|          24.0|  2368|
|          25.0|  2118|
|          26.0|  1899|
|          27.0|  1757|
|          28.0|  1498|
|          29.0|  1274|
|          30.0|  1145|
|          31.0|   933|
|          32.0|   825|
|          33.0|   730|
|          34.0|   594|
|          35.0|   490|
|          36.0|   412|
|          37.0|

In [57]:
df_transactions_flat.filter(col("amount") < 1).groupBy("amount").count().orderBy("amount").show(100)


+------+-----+
|amount|count|
+------+-----+
|  0.05|  431|
|  0.06|  109|
|  0.07|   89|
|  0.08|  100|
|  0.09|   87|
|   0.1|   95|
|  0.11|   99|
|  0.12|   79|
|  0.13|   97|
|  0.14|  110|
|  0.15|  110|
|  0.16|  108|
|  0.17|  113|
|  0.18|  108|
|  0.19|  109|
|   0.2|  105|
|  0.21|  119|
|  0.22|  120|
|  0.23|  123|
|  0.24|  118|
|  0.25|   96|
|  0.26|  128|
|  0.27|  117|
|  0.28|  120|
|  0.29|  122|
|   0.3|  126|
|  0.31|  116|
|  0.32|  111|
|  0.33|  112|
|  0.34|   98|
|  0.35|  130|
|  0.36|  148|
|  0.37|  126|
|  0.38|  127|
|  0.39|  128|
|   0.4|  140|
|  0.41|  136|
|  0.42|  134|
|  0.43|  142|
|  0.44|  138|
|  0.45|  121|
|  0.46|  146|
|  0.47|  133|
|  0.48|  143|
|  0.49|  138|
|   0.5|  159|
|  0.51|  141|
|  0.52|  136|
|  0.53|  159|
|  0.54|  129|
|  0.55|  147|
|  0.56|  134|
|  0.57|  143|
|  0.58|  131|
|  0.59|  147|
|   0.6|  144|
|  0.61|  126|
|  0.62|  123|
|  0.63|  142|
|  0.64|  145|
|  0.65|  152|
|  0.66|  166|
|  0.67|  152|
|  0.68|  

In [48]:
lim_amount = calcular_outliers(df_transactions_flat, "amount")
lim_reward = calcular_outliers(df_transactions_flat, "reward")


📌 AMOUNT
Q1: 2.69, Q3: 18.08, IQR: 15.389999999999999
Limite inferior: -20.394999999999996, Limite superior: 41.16499999999999
📌 REWARD
Q1: 2.0, Q3: 5.0, IQR: 3.0
Limite inferior: -2.5, Limite superior: 9.5


In [ ]:
from pyspark.sql.functions import col, round

# Arredonda os valores de amount pra facilitar análise
df_valores_agrupados = df_transactions_flat \
    .withColumn("amount_rounded", round(col("amount"), 0)) \
    .groupBy("amount_rounded") \
    .count() \
    .orderBy("amount_rounded")

df_valores_agrupados.show(100)


In [64]:
from pyspark.sql.functions import col, round

# Arredonda os valores de reward pra facilitar análise
df_valores_agrupados = df_transactions_flat \
    .withColumn("reward_rounded", round(col("reward"), 0)) \
    .groupBy("reward_rounded") \
    .count() \
    .orderBy("reward_rounded")

df_valores_agrupados.show(100)


+--------------+------+
|reward_rounded| count|
+--------------+------+
|          null|272955|
|           2.0|  9334|
|           3.0|  5156|
|           5.0| 12070|
|          10.0|  7019|
+--------------+------+



In [66]:
df_transactions_flat.show(5)

+--------------------+--------------+---------------------+------+------+--------------------+---------+--------------------+
|          account_id|         event|time_since_test_start|amount|reward|            offer_id|_offer_id|      final_offer_id|
+--------------------+--------------+---------------------+------+------+--------------------+---------+--------------------+
|78afa995795e4d85b...|offer received|                  0.0|  null|  null|9b98b8c7a33c4b65b...|     null|9b98b8c7a33c4b65b...|
|a03223e636434f42a...|offer received|                  0.0|  null|  null|0b1e1539f2cc45b7b...|     null|0b1e1539f2cc45b7b...|
|e2127556f4f64592b...|offer received|                  0.0|  null|  null|2906b810c7d441179...|     null|2906b810c7d441179...|
|8ec6ce2a7e7949b1b...|offer received|                  0.0|  null|  null|fafdcd668e3743c1b...|     null|fafdcd668e3743c1b...|
|68617ca6246f4fbc8...|offer received|                  0.0|  null|  null|4d5c57ea9a6940dd8...|     null|4d5c57ea9a6940

# 2. Enriquecimento da tabela `df_transactions_flat` com a `df_offers`

**Objetivo**
- Enriquecer os eventos transacionais com os dados da oferta (tipo, valor, canal, duração, etc).
- Verificar se os valores cruzados fazem sentido.
- Entender qual tipo de oferta foi recebida, completada ou visualizada.
- Calcular taxas de sucesso por tipo de cupom, canal ou perfil de cliente.

**Conclusão**
- há perda de conversão ao longo do funil. Nem todos que recebem a oferta visualizam, e menos ainda completam.

## 2.1. Verificar se todos os final_offer_id estão presentes na tabela offers

In [82]:
ids_transactions = df_transactions_flat.select("final_offer_id").distinct()
ids_offers = df_offers.select(col("id").alias("offer_id")).distinct()

ids_not_matched = ids_transactions.join(
    ids_offers,
    ids_transactions.final_offer_id == ids_offers.offer_id,
    how="left_anti"
)

ids_not_matched.show()


+--------------+
|final_offer_id|
+--------------+
|          null|
+--------------+



> *Join entre transações e ofertas com base no ID final da oferta*

In [92]:
# Renomeando a coluna 'id' da tabela de ofertas para evitar conflito
df_offers_renamed = df_offers.withColumnRenamed("id", "offer_metadata_id")

df_joined = df_transactions_flat.join(
    df_offers_renamed,
    df_transactions_flat.final_offer_id == df_offers_renamed.offer_metadata_id,
    how="left"
)


In [93]:
df_joined.select(
    "event", "final_offer_id", "offer_type", "discount_value", "channels", "min_value"
).where("final_offer_id IS NOT NULL").show(10, truncate=False)


+--------------+--------------------------------+-------------+--------------+----------------------------+---------+
|event         |final_offer_id                  |offer_type   |discount_value|channels                    |min_value|
+--------------+--------------------------------+-------------+--------------+----------------------------+---------+
|offer received|9b98b8c7a33c4b65b9aebfe6a799e6d9|bogo         |5             |[web, email, mobile]        |5        |
|offer received|0b1e1539f2cc45b7b9fa7c272da2e1d7|discount     |5             |[web, email]                |20       |
|offer received|2906b810c7d4411798c6938adc9daaa5|discount     |2             |[web, email, mobile]        |10       |
|offer received|fafdcd668e3743c1bb461111dcafc2a4|discount     |2             |[web, email, mobile, social]|10       |
|offer received|4d5c57ea9a6940dd891ad53e9dbe8da0|bogo         |10            |[web, email, mobile, social]|10       |
|offer received|f19421c1d4aa40978ebb69ca19b0e20d|bogo   

## 2.2. Validar o Join com offers

> *Ter certeza de que todos os 10 IDs únicos de oferta foram integrados corretamente após o merge*

In [94]:
# Valida se todos os final_offer_id conseguiram fazer join com dados da tabela offers
df_joined.select("final_offer_id").distinct().count(), df_offers.select("id").distinct().count()


(11, 10)

## 2.3. Contar quantos eventos foram enriquecidos por tipo

> *Confirma se o merge respeitou a lógica dos eventos que possuem offer_id original*

In [95]:
# Conta eventos com informações completas por tipo
df_joined.filter(col("final_offer_id").isNotNull()) \
    .groupBy("event").count().show()


+---------------+-----+
|          event|count|
+---------------+-----+
| offer received|76277|
|offer completed|33579|
|   offer viewed|57725|
+---------------+-----+



## 2.4. Análise exploratória dos canais por tipo de oferta

> *pode revelar estratégias de veiculação por tipo de campanha (e.g., será que informational usa menos canais que bogo?)*

In [100]:
from pyspark.sql.functions import size

df_joined = df_joined.withColumn("channel_count", size(col("channels")))

# Analisar média de canais por tipo de oferta
df_joined.filter(col("final_offer_id").isNotNull()) \
    .groupBy("offer_type") \
    .agg(
        count("*").alias("total"),
        avg("channel_count").alias("avg_channel_count"),
        expr("percentile(channel_count, array(0.25, 0.5, 0.75))").alias("channel_count_percentiles")
    ).show(truncate=False)


+-------------+-----+------------------+-------------------------+
|offer_type   |total|avg_channel_count |channel_count_percentiles|
+-------------+-----+------------------+-------------------------+
|discount     |69898|3.3809694125726057|[3.0, 4.0, 4.0]          |
|informational|26066|3.0               |[3.0, 3.0, 3.0]          |
|bogo         |71617|3.521566108605499 |[3.0, 4.0, 4.0]          |
+-------------+-----+------------------+-------------------------+



## 2.5. Análise de discount_value e min_value por tipo de oferta

> *Pode revelar, por exemplo, que bogo exige compras maiores, mas tem desconto maior*

- discount: desconto médio de ~2,88 e min_value médio de ~11,10
- bogo: desconto e min_value iguais (~7,53), pois esse tipo exige gasto para ativar.
- informational: ambos os valores são 0, já que são apenas comunicados, sem incentivo financeiro.

**Hipótese**
- valores mínimos mais altos podem impactar negativamente a taxa de conversão em grupos menos engajados.

In [101]:

df_joined.filter(col("final_offer_id").isNotNull()) \
    .groupBy("offer_type") \
    .agg(
        avg("discount_value").alias("avg_discount"),
        avg("min_value").alias("avg_min_value"),
        expr("percentile(min_value, array(0.25, 0.5, 0.75))").alias("min_value_percentiles")
    ).show(truncate=False)


+-------------+-----------------+------------------+---------------------+
|offer_type   |avg_discount     |avg_min_value     |min_value_percentiles|
+-------------+-----------------+------------------+---------------------+
|discount     |2.878308392228676|11.102935706314916|[7.0, 10.0, 10.0]    |
|informational|0.0              |0.0               |[0.0, 0.0, 0.0]      |
|bogo         |7.533197425192343|7.533197425192343 |[5.0, 10.0, 10.0]    |
+-------------+-----------------+------------------+---------------------+



# 3. Enriquecimento da tabela `df_joined` com a `df_customers_clean `

**Objetivo**

- Analisar se existem taxas de conversão por faixa etária ou limite de cartão.
- análise de uso de canais por perfil.
- segmentações mais eficientes para o modelo ou estratégias.


**Conclusão**

- cerca de 12,8% dos clientes ativos não têm dados demográficos disponíveis. Podemos considerar excluir esses registros de análises que dependam de atributos do cliente (como idade ou limite), ou tratá-los separadamente caso façam parte de outra estratégia (cold start)

In [103]:
df_customers_clean = df_customers_clean.withColumnRenamed("id", "customer_id")

df_final = df_joined.join(
    df_customers_clean,
    df_joined.account_id == df_customers_clean.customer_id,
    how="left"
)


## 3.1. Verificar se todos os customer_id foram encontrados

In [107]:
clientes_ativos = df_joined.select("account_id").distinct().count()
clientes_ativos


17000

## 3.2. Verificar quantos desses clientes receberam dados no df_final

In [106]:
clientes_com_dados = df_final.filter(col("age").isNotNull()).select("account_id").distinct().count()
print(f"Clientes com dados completos (df_final): {clientes_com_dados}")


Clientes com dados completos (df_final): 14825


# 4. Análises rapidas do comportamento dos clientes

In [119]:
df_behavior = df_final.groupBy("account_id") \
    .pivot("event", ["offer completed", "offer received", "offer viewed", "transaction"]) \
    .count() \
    .fillna(0)


## 4.1. Taxa de visualização e conclusão por cliente

> **Hipótese 1:** *agrupar clientes com comportamento ideal de engajamento e usar esse grupo como benchmark para entender o que eles têm em comum: idade, limite de cartão, canal preferido, tipo de oferta etc.*

> **Hipótese 2:** *clientes com view_rate alta e completion_rate baixa. Pode indicar que as ofertas não eram atraentes mesmo sendo vistas. Talvez o valor mínimo era alto demais para o perfil do cliente.*
> - "Mesmo clientes que visualizam as ofertas não estão completando algumas campanhas — nossa proposta inclui adaptar o valor mínimo por perfil de consumo."

> **Hipótese 3:** *Analisar se quem mais completa ofertas também transaciona mais. indicando que o cupom não apenas atrai, mas fideliza ou aumenta volume de compras.*

> **Hipótese 3:** *talvez uma simples regra baseada nesses indicadores para definir grupos como:*

| Segmento             | Critério                                    |
| -------------------- | ------------------------------------------- |
| Engajado             | `view_rate` > 0.8 e `completion_rate` > 0.8 |
| Visualizador passivo | `view_rate` > 0.8 e `completion_rate` < 0.5 |
| Desconectado         | `view_rate` < 0.5                           |


*Ex: Dos 17 mil clientes, 68% visualizam ao menos uma oferta e 48% chegam a concluir. Porém, apenas 33% fazem isso de forma consistente. Nossa proposta é aumentar essa taxa com segmentações mais direcionadas.*

In [121]:
from pyspark.sql.functions import col, when, round

df_behavior = df_behavior \
    .withColumn("view_rate", round(col("offer viewed") / when(col("offer received") > 0, col("offer received")), 2)) \
    .withColumn("completion_rate", round(col("offer completed") / when(col("offer viewed") > 0, col("offer viewed")), 2))


In [122]:
df_behavior.show()

+--------------------+---------------+--------------+------------+-----------+---------+---------------+
|          account_id|offer completed|offer received|offer viewed|transaction|view_rate|completion_rate|
+--------------------+---------------+--------------+------------+-----------+---------+---------------+
|8ca462f1192b4aa29...|              0|             4|           3|         13|     0.75|            0.0|
|5abe3df001c14294a...|              4|             5|           5|         10|      1.0|            0.8|
|5e0cac0673884c67b...|              0|             4|           2|          8|      0.5|            0.0|
|74450de8dc654737b...|              1|             2|           2|          4|      1.0|            0.5|
|d3c24fa42d0947a4b...|              3|             4|           2|         15|      0.5|            1.5|
|dd1069bbc7ef423c9...|              1|             6|           5|         19|     0.83|            0.2|
|b3d1075ec03c4c188...|              4|             4|  

## 5. Criação do df final que será utilizado nos modelos

**Objetivo**
- Criar um dataset que representa, para cada cliente, um resumo estratégico do seu:
  - Engajamento com cupons
  - Comportamento de compra
  - Preferência por tipo de oferta e canal
  - Perfil demográfico e histórico na plataforma

A intenção é que essa estrutura permita capturar os fatores mais relevantes que influenciam o sucesso ou o fracasso de uma campanha de cupons.

- Pontos que planejo responder com esse dataset + modelos:
  - Se o cliente vai completar uma próxima oferta (classificação binária)
  - Qual o retorno estimado de uma campanha para aquele cliente (regressão)
  - Qual cliente mais se beneficiaria de uma oferta específica (uplift modeling)



> *Engajamento com Ofertas*

In [126]:
from pyspark.sql.functions import count, expr, when

oferta_engajamento = df_final.filter(col("final_offer_id").isNotNull()) \
    .groupBy("account_id").agg(
        count(when(col("event") == "offer received", True)).alias("offer_received_count"),
        count(when(col("event") == "offer viewed", True)).alias("offer_viewed_count"),
        count(when(col("event") == "offer completed", True)).alias("offer_completed_count")
    ).withColumn("view_rate", expr("offer_viewed_count / offer_received_count")) \
     .withColumn("completion_rate", expr("offer_completed_count / offer_viewed_count"))


In [127]:
oferta_engajamento.show(5)

+--------------------+--------------------+------------------+---------------------+------------------+---------------+
|          account_id|offer_received_count|offer_viewed_count|offer_completed_count|         view_rate|completion_rate|
+--------------------+--------------------+------------------+---------------------+------------------+---------------+
|c05652fa9d2c4fac9...|                   6|                 4|                    1|0.6666666666666666|           0.25|
|9da3f2339b244229a...|                   5|                 2|                    1|               0.4|            0.5|
|fc58860830e648c9b...|                   4|                 2|                    0|               0.5|            0.0|
|f608dc84f5f744a88...|                   5|                 3|                    3|               0.6|            1.0|
|25540a042d7d4aabb...|                   3|                 2|                    1|0.6666666666666666|            0.5|
+--------------------+------------------

> *Comportamento Transacional*

In [128]:
from pyspark.sql.functions import sum as spark_sum, avg, stddev

transacional = df_final.filter(col("event") == "transaction") \
    .groupBy("account_id").agg(
        count("*").alias("transaction_count"),
        spark_sum("amount").alias("total_amount_spent"),
        avg("amount").alias("avg_transaction_amount"),
        stddev("amount").alias("std_transaction_amount")
    )


In [130]:
transacional.show(5)

+--------------------+-----------------+------------------+----------------------+----------------------+
|          account_id|transaction_count|total_amount_spent|avg_transaction_amount|std_transaction_amount|
+--------------------+-----------------+------------------+----------------------+----------------------+
|08579355f46b49efa...|                6|            134.99|    22.498333333333335|     8.106433042137994|
|708d8271f758417e9...|               14|             59.99|                 4.285|     2.516886813261421|
|dd1069bbc7ef423c9...|               19|61.120000000000005|     3.216842105263158|     2.578166990177278|
|9da3f2339b244229a...|                4|             18.87|                4.7175|     2.487105077528223|
|30105215903d4eefb...|               12|            139.85|    11.654166666666667|    4.5565566312000705|
+--------------------+-----------------+------------------+----------------------+----------------------+
only showing top 5 rows



> *Preferência por Tipo de Oferta*

In [131]:
preferencias = df_final.filter(col("event") == "offer completed") \
    .groupBy("account_id").pivot("offer_type", ["bogo", "discount", "informational"]).agg(count("*")) \
    .na.fill(0).withColumnRenamed("bogo", "completed_bogo") \
    .withColumnRenamed("discount", "completed_discount") \
    .withColumnRenamed("informational", "completed_info") \
    .withColumn(
        "preferred_offer_type",
        when((col("completed_bogo") >= col("completed_discount")) & (col("completed_bogo") >= col("completed_info")), "bogo")
        .when((col("completed_discount") >= col("completed_bogo")) & (col("completed_discount") >= col("completed_info")), "discount")
        .otherwise("informational")
    )


In [132]:
preferencias.show(5)

+--------------------+--------------+------------------+--------------+--------------------+
|          account_id|completed_bogo|completed_discount|completed_info|preferred_offer_type|
+--------------------+--------------+------------------+--------------+--------------------+
|559943eeb754463e8...|             1|                 0|             0|                bogo|
|72c8dfba6eb44158a...|             3|                 0|             0|                bogo|
|f5de20984950433d9...|             4|                 0|             0|                bogo|
|304183fd053441f0a...|             3|                 1|             0|                bogo|
|9da3f2339b244229a...|             0|                 1|             0|            discount|
+--------------------+--------------+------------------+--------------+--------------------+
only showing top 5 rows



> *Canais Utilizados*

In [133]:

from pyspark.sql.functions import array_contains, size, max as spark_max

canal_flags = df_final.filter(col("event") == "offer received") \
    .withColumn("channel_web", when(array_contains(col("channels"), "web"), 1).otherwise(0)) \
    .withColumn("channel_email", when(array_contains(col("channels"), "email"), 1).otherwise(0)) \
    .withColumn("channel_mobile", when(array_contains(col("channels"), "mobile"), 1).otherwise(0)) \
    .groupBy("account_id").agg(
        spark_max("channel_web").alias("channel_web"),
        spark_max("channel_email").alias("channel_email"),
        spark_max("channel_mobile").alias("channel_mobile"),
        avg(size(col("channels"))).alias("avg_channel_count")
    )


In [134]:
canal_flags.show(5)

+--------------------+-----------+-------------+--------------+------------------+
|          account_id|channel_web|channel_email|channel_mobile| avg_channel_count|
+--------------------+-----------+-------------+--------------+------------------+
|c05652fa9d2c4fac9...|          1|            1|             1|3.1666666666666665|
|9da3f2339b244229a...|          1|            1|             1|               3.0|
|fc58860830e648c9b...|          1|            1|             1|              3.25|
|f608dc84f5f744a88...|          1|            1|             1|               3.2|
|25540a042d7d4aabb...|          1|            1|             1|3.3333333333333335|
+--------------------+-----------+-------------+--------------+------------------+
only showing top 5 rows



> *Perfil Demográfico*

In [135]:
clientes_demo = df_final.select("account_id", "age", "credit_card_limit", "days_since_registration") \
    .dropna(subset=["age", "credit_card_limit", "days_since_registration"]).dropDuplicates(["account_id"])


In [136]:
clientes_demo.show(5)

+--------------------+---+-----------------+-----------------------+
|          account_id|age|credit_card_limit|days_since_registration|
+--------------------+---+-----------------+-----------------------+
|c05652fa9d2c4fac9...| 62|          73000.0|                   2657|
|9da3f2339b244229a...| 49|          36000.0|                   2675|
|fc58860830e648c9b...| 79|          64000.0|                   2520|
|f608dc84f5f744a88...| 68|         115000.0|                   2927|
|25540a042d7d4aabb...| 71|         108000.0|                   2715|
+--------------------+---+-----------------+-----------------------+
only showing top 5 rows



> *Final*

In [137]:
df_final_analytics = df_final.select("account_id").distinct() \
    .join(oferta_engajamento, on="account_id", how="left") \
    .join(transacional, on="account_id", how="left") \
    .join(preferencias, on="account_id", how="left") \
    .join(canal_flags, on="account_id", how="left") \
    .join(clientes_demo, on="account_id", how="left")


In [138]:
df_final_analytics.show(5)

+--------------------+--------------------+------------------+---------------------+------------------+---------------+-----------------+------------------+----------------------+----------------------+--------------+------------------+--------------+--------------------+-----------+-------------+--------------+------------------+---+-----------------+-----------------------+
|          account_id|offer_received_count|offer_viewed_count|offer_completed_count|         view_rate|completion_rate|transaction_count|total_amount_spent|avg_transaction_amount|std_transaction_amount|completed_bogo|completed_discount|completed_info|preferred_offer_type|channel_web|channel_email|channel_mobile| avg_channel_count|age|credit_card_limit|days_since_registration|
+--------------------+--------------------+------------------+---------------------+------------------+---------------+-----------------+------------------+----------------------+----------------------+--------------+------------------+------

In [145]:
os.makedirs("ifood-case/data/processed", exist_ok=True)

In [146]:
output_path = "ifood-case/data/processed/df_final_analytics.parquet"
df_final_analytics.write.mode("overwrite").parquet(output_path)

In [147]:
!ls -l ifood-case/data/processed/


total 4
drwxr-xr-x 2 root root 4096 May  7 16:51 df_final_analytics.parquet


In [149]:
!zip -r df_final_analytics.zip ifood-case/data/processed/df_final_analytics.parquet

from google.colab import files
files.download("df_final_analytics.zip")


  adding: ifood-case/data/processed/df_final_analytics.parquet/ (stored 0%)
  adding: ifood-case/data/processed/df_final_analytics.parquet/.part-00000-f18fb514-fd07-44c7-a67f-e7a1b24d1fbc-c000.snappy.parquet.crc (stored 0%)
  adding: ifood-case/data/processed/df_final_analytics.parquet/._SUCCESS.crc (stored 0%)
  adding: ifood-case/data/processed/df_final_analytics.parquet/_SUCCESS (stored 0%)
  adding: ifood-case/data/processed/df_final_analytics.parquet/part-00000-f18fb514-fd07-44c7-a67f-e7a1b24d1fbc-c000.snappy.parquet (deflated 19%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [148]:
df_loaded = spark.read.parquet("ifood-case/data/processed/df_final_analytics.parquet")
df_loaded.show()


+--------------------+--------------------+------------------+---------------------+------------------+------------------+-----------------+------------------+----------------------+----------------------+--------------+------------------+--------------+--------------------+-----------+-------------+--------------+------------------+----+-----------------+-----------------------+
|          account_id|offer_received_count|offer_viewed_count|offer_completed_count|         view_rate|   completion_rate|transaction_count|total_amount_spent|avg_transaction_amount|std_transaction_amount|completed_bogo|completed_discount|completed_info|preferred_offer_type|channel_web|channel_email|channel_mobile| avg_channel_count| age|credit_card_limit|days_since_registration|
+--------------------+--------------------+------------------+---------------------+------------------+------------------+-----------------+------------------+----------------------+----------------------+--------------+--------------

---